In [50]:
pip install optuna

     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 210 kB 67.3 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 78 kB 8.2 MB/s 
     |████████████████████████████████| 49 kB 7.2 MB/s 
     |████████████████████████████████| 113 kB 74.7 MB/s 
     |████████████████████████████████| 150 kB 43.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=89c2f05762a9b3999877f4b8b5469fb2aa6032896013cf3bd4fb12188aa416a5
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [51]:
import numpy as np
import pandas as pd

import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import optuna
import xgboost as xgb

from sklearn.metrics import r2_score

In [52]:
data = pd.read_csv('/content/SolarPrediction.csv')

In [53]:
data

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00
32682,1480587301,12/1/2016 12:00:00 AM,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00
32683,1480587001,12/1/2016 12:00:00 AM,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00
32684,1480586702,12/1/2016 12:00:00 AM,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00


In [54]:
data.info

<bound method DataFrame.info of          UNIXTime                   Data      Time  Radiation  Temperature  \
0      1475229326  9/29/2016 12:00:00 AM  23:55:26       1.21           48   
1      1475229023  9/29/2016 12:00:00 AM  23:50:23       1.21           48   
2      1475228726  9/29/2016 12:00:00 AM  23:45:26       1.23           48   
3      1475228421  9/29/2016 12:00:00 AM  23:40:21       1.21           48   
4      1475228124  9/29/2016 12:00:00 AM  23:35:24       1.17           48   
...           ...                    ...       ...        ...          ...   
32681  1480587604  12/1/2016 12:00:00 AM  00:20:04       1.22           44   
32682  1480587301  12/1/2016 12:00:00 AM  00:15:01       1.17           44   
32683  1480587001  12/1/2016 12:00:00 AM  00:10:01       1.20           44   
32684  1480586702  12/1/2016 12:00:00 AM  00:05:02       1.23           44   
32685  1480586402  12/1/2016 12:00:00 AM  00:00:02       1.20           44   

       Pressure  Humidity  Wind

In [55]:
print("Total missing values:", data.isna().sum().sum())

Total missing values: 0


In [56]:
##feature_engineering

In [57]:
data['Month'] = data['Data'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(np.int)
data['Day'] = data['Data'].apply(lambda x: re.search(r'(?<=\/)\d+(?=\/)', x).group(0)).astype(np.int)
data['Year'] = data['Data'].apply(lambda x: re.search(r'(?<=\/)\d+(?=\s)', x).group(0)).astype(np.int)

data = data.drop('Data', axis=1)

In [58]:
data

,UNIXTime,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet,Month,Day,Year
0,1475229326,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00,9,29,2016
1,1475229023,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00,9,29,2016
2,1475228726,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00,9,29,2016
3,1475228421,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00,9,29,2016
4,1475228124,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00,9,29,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00,12,1,2016
32682,1480587301,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00,12,1,2016
32683,1480587001,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00,12,1,2016
32684,1480586702,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00,12,1,2016


In [59]:
data['Hour'] = data['Time'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(np.int)
data['Minute'] = data['Time'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(np.int)
data['Second'] = data['Time'].apply(lambda x: re.search(r'\d+$', x).group(0)).astype(np.int)

data = data.drop('Time', axis=1)

In [60]:
data

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet,Month,Day,Year,Hour,Minute,Second
0,1475229326,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00,9,29,2016,23,55,26
1,1475229023,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00,9,29,2016,23,50,23
2,1475228726,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00,9,29,2016,23,45,26
3,1475228421,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00,9,29,2016,23,40,21
4,1475228124,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00,9,29,2016,23,35,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00,12,1,2016,0,20,4
32682,1480587301,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00,12,1,2016,0,15,1
32683,1480587001,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00,12,1,2016,0,10,1
32684,1480586702,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00,12,1,2016,0,5,2


In [61]:
data['SunriseHour'] = data['TimeSunRise'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(np.int)
data['SunriseMinute'] = data['TimeSunRise'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(np.int)

data['SunsetHour'] = data['TimeSunSet'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(np.int)
data['SunsetMinute'] = data['TimeSunSet'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(np.int)

data = data.drop(['TimeSunRise', 'TimeSunSet'], axis=1)

In [62]:
data

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,Month,Day,Year,Hour,Minute,Second,SunriseHour,SunriseMinute,SunsetHour,SunsetMinute
0,1475229326,1.21,48,30.46,59,177.39,5.62,9,29,2016,23,55,26,6,13,18,13
1,1475229023,1.21,48,30.46,58,176.78,3.37,9,29,2016,23,50,23,6,13,18,13
2,1475228726,1.23,48,30.46,57,158.75,3.37,9,29,2016,23,45,26,6,13,18,13
3,1475228421,1.21,48,30.46,60,137.71,3.37,9,29,2016,23,40,21,6,13,18,13
4,1475228124,1.17,48,30.46,62,104.95,5.62,9,29,2016,23,35,24,6,13,18,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,1.22,44,30.43,102,145.42,6.75,12,1,2016,0,20,4,6,41,17,42
32682,1480587301,1.17,44,30.42,102,117.78,6.75,12,1,2016,0,15,1,6,41,17,42
32683,1480587001,1.20,44,30.42,102,145.19,9.00,12,1,2016,0,10,1,6,41,17,42
32684,1480586702,1.23,44,30.42,101,164.19,7.87,12,1,2016,0,5,2,6,41,17,42


In [63]:
data.dtypes

UNIXTime                    int64
Radiation                 float64
Temperature                 int64
Pressure                  float64
Humidity                    int64
WindDirection(Degrees)    float64
Speed                     float64
Month                       int64
Day                         int64
Year                        int64
Hour                        int64
Minute                      int64
Second                      int64
SunriseHour                 int64
SunriseMinute               int64
SunsetHour                  int64
SunsetMinute                int64
dtype: object

In [64]:
data['Year'].unique()

array([2016])

In [65]:
data['SunriseHour'].unique()

array([6])

In [66]:
data = data.drop(['Year', 'SunriseHour'], axis=1)

In [67]:
##splitting/scaling

In [68]:
y = data['Radiation'].copy()
X = data.drop('Radiation', axis=1).copy()

In [69]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=100)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=200)

In [71]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [72]:
##hyperparameter search

In [73]:
def get_model_rmse(params):
    model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'eval')], early_stopping_rounds=10, verbose_eval=0)
    results = model.eval(dval)
    rmse = np.float(re.search(r'[\d.]+$', results).group(0))
    return rmse

In [74]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 0.00001, 10.0)
    max_depth = trial.suggest_int('max_depth', 4, 8)
    l1_reg = trial.suggest_loguniform('l1_reg', 0.00001, 10.0)
    l2_reg = trial.suggest_loguniform('l2_reg', 0.00001, 10.0)
    
    params = {'learning_rate': learning_rate, 'max_depth': max_depth, 'alpha': l1_reg, 'lambda': l2_reg}
    
    return get_model_rmse(params)

In [75]:
study = optuna.create_study()
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2022-03-20 07:28:54,825] A new study created in memory with name: no-name-6a0516f9-d02d-48ca-937d-85539ce618b4


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-03-20 07:28:57,803] Trial 0 finished with value: 92.082428 and parameters: {'learning_rate': 0.05925032224653448, 'max_depth': 6, 'l1_reg': 0.00011131711839995555, 'l2_reg': 0.01650549815258624}. Best is trial 0 with value: 92.082428.
[I 2022-03-20 07:29:00,178] Trial 1 finished with value: 308.094269 and parameters: {'learning_rate': 0.0024830487131916787, 'max_depth': 7, 'l1_reg': 0.626129239617582, 'l2_reg': 7.299222700588932}. Best is trial 0 with value: 92.082428.
[I 2022-03-20 07:29:01,813] Trial 2 finished with value: 115.674126 and parameters: {'learning_rate': 0.024449468306436833, 'max_depth': 5, 'l1_reg': 0.008138083613503821, 'l2_reg': 1.0955234682170943e-05}. Best is trial 0 with value: 92.082428.
[I 2022-03-20 07:29:04,174] Trial 3 finished with value: 357.55188 and parameters: {'learning_rate': 0.0007210637631351631, 'max_depth': 7, 'l1_reg': 2.0930381710192996e-05, 'l2_reg': 2.4044891223642495e-05}. Best is trial 0 with value: 92.082428.
[I 2022-03-20 07:29:06,6

In [76]:
best_params = study.best_params
best_params

{'l1_reg': 2.809899935745351e-05,
 'l2_reg': 1.4803702129192733,
 'learning_rate': 0.12741212508728628,
 'max_depth': 8}

In [77]:
model = xgb.train(best_params, dtrain, num_boost_round=10000, evals=[(dval, 'eval')], early_stopping_rounds=10)

[0]	eval-rmse:337.546
Will train until eval-rmse hasn't improved in 10 rounds.
[1]	eval-rmse:299.504
[2]	eval-rmse:266.814
[3]	eval-rmse:238.645
[4]	eval-rmse:214.131
[5]	eval-rmse:193.644
[6]	eval-rmse:176.163
[7]	eval-rmse:161.185
[8]	eval-rmse:148.472
[9]	eval-rmse:137.774
[10]	eval-rmse:128.896
[11]	eval-rmse:121.507
[12]	eval-rmse:115.396
[13]	eval-rmse:110.546
[14]	eval-rmse:106.304
[15]	eval-rmse:103.213
[16]	eval-rmse:100.484
[17]	eval-rmse:98.163
[18]	eval-rmse:96.3431
[19]	eval-rmse:94.8992
[20]	eval-rmse:93.4228
[21]	eval-rmse:91.9602
[22]	eval-rmse:90.9774
[23]	eval-rmse:90.2548
[24]	eval-rmse:89.5052
[25]	eval-rmse:89.0271
[26]	eval-rmse:88.541
[27]	eval-rmse:87.9364
[28]	eval-rmse:87.6868
[29]	eval-rmse:87.417
[30]	eval-rmse:87.235
[31]	eval-rmse:87.0299
[32]	eval-rmse:86.8749
[33]	eval-rmse:86.7336
[34]	eval-rmse:86.5809
[35]	eval-rmse:86.3732
[36]	eval-rmse:86.3067
[37]	eval-rmse:86.1582
[38]	eval-rmse:86.073
[39]	eval-rmse:85.9434
[40]	eval-rmse:85.8961
[41]	eval-rmse:

In [78]:
##results

In [79]:

y_true = np.array(y_test, dtype=np.float)
y_pred = np.array(model.predict(dtest), dtype=np.float)

In [80]:
r2 = r2_score(y_true, y_pred)

print("R^2 Score: {:.4f}".format(r2))

R^2 Score: 0.9412
